In [1]:
import pandas as pd

# 1. Cargamos tus leads clasificados (los 152 de Oro)
df_leads = pd.read_parquet('../users_with_target_week1.parquet')

# 2. Cargamos el catálogo de caballos que ya limpiaron tus compañeros
df_caballos = pd.read_parquet('../equinenow_horses_listings_limpio.parquet')

print("✅ ¡Conexión exitosa!")
print(f"Tienes {len(df_leads)} usuarios listos y {len(df_caballos)} caballos para ofrecer.")

✅ ¡Conexión exitosa!
Tienes 200000 usuarios listos y 1905 caballos para ofrecer.


In [14]:
def motor_recomendacion_simple(row):
    # Definimos el nombre correcto de la columna que descubrimos
    col_p = 'Price' 
    
    # Regla para Lead ORO: Recomendamos los caballos más caros (Top 3)
    if row['target_lead'] == 'Oro':
        top_caballos = df_caballos.nlargest(3, col_p)['Name'].tolist()
        return f"VIP: Te recomendamos estos ejemplares exclusivos: {', '.join(top_caballos)}"
    
    # Regla para Lead PLATA: Recomendamos caballos de precio medio
    elif row['target_lead'] == 'Plata':
        # Filtramos y tomamos los primeros 3 que cumplan el rango
        filtro_plata = df_caballos[(df_caballos[col_p] > 10000) & (df_caballos[col_p] <= 50000)]
        medio_caballos = filtro_plata.head(3)['Name'].tolist()
        return f"Calidad: Mira estos caballos para tu nivel: {', '.join(medio_caballos)}"
    
    # Regla para INTERESADOS (y Bronce si apareciera): Los más económicos
    else:
        barato_caballos = df_caballos.nsmallest(3, col_p)['Name'].tolist()
        return f"Inicio: Empieza tu aventura con: {', '.join(barato_caballos)}"

# 1. Ejecutamos el motor con una muestra de 10
df_muestra = df_leads.sample(10) 
df_muestra['recomendacion'] = df_muestra.apply(motor_recomendacion_simple, axis=1)

# 2. Mostramos los resultados de los caballos
print("--- RECOMENDACIONES DE CABALLOS ---")
display(df_muestra[['target_lead', 'recomendacion']])

# 3. Agregamos lo de los productos (Accesorios)
print("\n--- RECOMENDACIONES DE PRODUCTOS ---")
df_productos = pd.read_parquet('doversaddlery_products_listing_limpio.parquet')
accesorios = df_productos.head(3)['Name'].tolist()
print(f"🛍️ Además, para tu nuevo equipo te sugerimos: {', '.join(accesorios)}")

--- RECOMENDACIONES DE CABALLOS ---


,target_lead,recomendacion
48562,Interesado,"Inicio: Empieza tu aventura con: vinnie, rosie..."
65797,Interesado,"Inicio: Empieza tu aventura con: vinnie, rosie..."
170038,Interesado,"Inicio: Empieza tu aventura con: vinnie, rosie..."
1780,Interesado,"Inicio: Empieza tu aventura con: vinnie, rosie..."
91807,Interesado,"Inicio: Empieza tu aventura con: vinnie, rosie..."
71984,Plata,Calidad: Mira estos caballos para tu nivel: ba...
25400,Interesado,"Inicio: Empieza tu aventura con: vinnie, rosie..."
58981,Interesado,"Inicio: Empieza tu aventura con: vinnie, rosie..."
120962,Interesado,"Inicio: Empieza tu aventura con: vinnie, rosie..."
83584,Interesado,"Inicio: Empieza tu aventura con: vinnie, rosie..."



--- RECOMENDACIONES DE PRODUCTOS ---
🛍️ Además, para tu nuevo equipo te sugerimos: troxel® sport 2.0™ helmet, weatherbeeta® free standard neck turnout sheet, rider’s international™ laced rein dog collar


In [13]:
df_productos.info()

<class 'pandas.DataFrame'>
RangeIndex: 937 entries, 0 to 936
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Item_ID      937 non-null    str    
 1   Name         937 non-null    str    
 2   Stock        937 non-null    int64  
 3   Description  937 non-null    str    
 4   Price        937 non-null    float64
 5   Images       937 non-null    str    
 6   URL          937 non-null    str    
dtypes: float64(1), int64(1), str(5)
memory usage: 721.3 KB
